In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [3]:
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)

In [6]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [7]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [8]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [11]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [13]:
inputs = Input(shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(128)(layer)
layer = Dense(128)(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1)(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 128)               91648     
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129   

In [14]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [15]:
history = model.fit(sequences_matrix,Y_train,batch_size=0,epochs=10,
          validation_split=0.2)

Epoch 1/10
112/112 [==============================] - 31s 239ms/step - loss: 0.2073 - accuracy: 0.9293 - val_loss: 0.0458 - val_accuracy: 0.9877
Epoch 2/10
112/112 [==============================] - 29s 255ms/step - loss: 0.0385 - accuracy: 0.9899 - val_loss: 0.0429 - val_accuracy: 0.9899
Epoch 3/10
112/112 [==============================] - 26s 235ms/step - loss: 0.0262 - accuracy: 0.9919 - val_loss: 0.0572 - val_accuracy: 0.9821
Epoch 4/10
112/112 [==============================] - 27s 245ms/step - loss: 0.0172 - accuracy: 0.9944 - val_loss: 0.0447 - val_accuracy: 0.9933
Epoch 5/10
112/112 [==============================] - 27s 241ms/step - loss: 0.0095 - accuracy: 0.9972 - val_loss: 0.0612 - val_accuracy: 0.9910
Epoch 6/10
112/112 [==============================] - 27s 244ms/step - loss: 0.0066 - accuracy: 0.9978 - val_loss: 0.0685 - val_accuracy: 0.9865
Epoch 7/10
112/112 [==============================] - 26s 232ms/step - loss: 0.0087 - accuracy: 0.9980 - val_loss: 0.0614 - val_ac

In [16]:
metrics = pd.DataFrame(history.history)
metrics.rename(columns = {'loss': 'Training_Loss', 'accuracy': 'Training_Accuracy', 'val_loss': 'Validation_Loss', 'val_accuracy': 'Validation_Accuracy'}, inplace = True)
def plot_graphs1(var1, var2, string):
    metrics[[var1, var2]].plot()
    plt.title('Training and Validation ' + string)
    plt.xlabel ('Number of epochs')
    plt.ylabel(string)
    plt.legend([var1, var2])

In [17]:
model.save('Spam_sms_classifier.h5')

In [20]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [21]:
accuracy1 = model.evaluate(test_sequences_matrix,Y_test)

35/35 [==============================] - 4s 114ms/step - loss: 0.1471 - accuracy: 0.9776


In [22]:
print(' Accuracy: {:0.3f}'.format(accuracy1[0],accuracy1[1]))

 Accuracy: 0.147
